In [2]:
import pyspark
from pyspark.sql import SparkSession

SPARK_VER = pyspark.__version__  

spark = (SparkSession.builder
         .appName("ev-stream")
         .config("spark.jars.packages", f"org.apache.spark:spark-sql-kafka-0-10_2.13:{SPARK_VER}")
         .getOrCreate())
spark.sparkContext.setLogLevel("WARN")


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/14 14:21:51 WARN Utils: Your hostname, Alona, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/01/14 14:21:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/alona/pyspark/venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/alona/.ivy2.5.2/cache
The jars for the packages stored in: /home/alona/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5e84c2b8-0737-4dc1-b597-fc7a1f416f39;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.1.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;4.1.1 in central
	found org.apache.kafka#kafka-clients;3.9.1 in central
	found org.lz4#l

In [6]:
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType

kafka_df = (spark.readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "localhost:9092")
  .option("subscribe", "ev_population")
  .option("startingOffsets", "earliest")
  .load())

schema = StructType([
    StructField("City", StringType(), True),
    StructField("County", StringType(), True),
    StructField("State", StringType(), True),
    StructField("Model Year", StringType(), True),  
])

parsed = (kafka_df
  .selectExpr("CAST(value AS STRING) AS value")
  .select(from_json(col("value"), schema).alias("data"))
  .select("data.*"))


In [5]:
from pyspark.sql.functions import col, desc

top3 = (parsed
  .filter(col("Model Year") == "2023")
  .groupBy("City")
  .count()
  .orderBy(desc("count"))
  .limit(3)
)

q2 = (top3.writeStream
      .format("console")
      .outputMode("complete")
      .option("truncate","false")
      .option("numRows", 3)
      .option("checkpointLocation","/tmp/ev_top3_chk2")
      .start())

q2


26/01/14 14:23:58 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


26/01/14 14:23:58 WARN MicroBatchExecution: Disabling AQE since AQE is not supported in stateful workloads.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+--------+-----+
|City    |count|
+--------+-----+
|Seattle |5610 |
|Bellevue|2088 |
|Tukwila |1773 |
+--------+-----+

